In [ ]:
import sagemaker

session = sagemaker.Session()
bucket = "sagemaker-subean"
role = sagemaker.get_execution_role()

In [ ]:
import tensorflow as tf
print(tf.__version__)  
tf_framework_version = tf.__version__

In [ ]:
import tensorflow as tf
import tarfile
import os

tf.keras.backend.set_image_data_format("channels_last")
pretrained_model = tf.keras.applications.resnet50.ResNet50()
saved_model_dir = "1"
tf.saved_model.save(pretrained_model, saved_model_dir)

In [ ]:
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add(saved_model_dir)

In [ ]:
!saved_model_cli show --all --dir ./1

In [ ]:
from sagemaker.utils import name_from_base

compilation_job_name = name_from_base("reKeras-ResNet50")
input_model_path = session.upload_data(
    path="model.tar.gz", bucket=bucket, key_prefix=compilation_job_name
)
print("S3 path for input model: {}".format(input_model_path))


In [ ]:
from sagemaker.tensorflow import TensorFlowModel

# Create a TensorFlow SageMaker model
tensorflow_model = TensorFlowModel(model_data=input_model_path, role=role, framework_version='2.3')

# Compile the model for EI accelerator in SageMaker Neo
output_path = "/".join(input_model_path.split("/")[:-1])


In [ ]:
original_predictor = tensorflow_model.deploy(
    initial_instance_count=1, instance_type="ml.m5.xlarge", accelerator_type="ml.eia2.large"
)

In [ ]:
%%time
import numpy as np

random_input = np.random.rand(1, 224, 224, 3)
prediction = original_predictor.predict({"inputs": random_input.tolist()})
print(prediction)

In [ ]:
optimize_model = tensorflow_model.compile(
    target_instance_family="ml_eia2",
    input_shape={"input_1": [1, 224, 224, 3]},
    output_path=output_path,
    role=role,
    job_name=compilation_job_name,
    framework="tensorflow",
)

In [ ]:
optimize_predictor = optimize_model.deploy(
    initial_instance_count=1, instance_type="ml.m5.xlarge", accelerator_type="ml.eia2.large"
)

In [ ]:
prediction = optimize_predictor.predict({"inputs": random_input.tolist()})